## Описание проекта


Для исследования применимости алгоритмов машинного обучения в задаче расчёта высоковакуумного турбомолекулярного насоса следует решить следующие задачи:

- Решить оптимизационную задачу нахождения минимального диаметра колеса ТМН;
- Сгенерировать данные и провести по ним EDA;
- Исследовать различные модели машинного обучения на предмет предсказания минимального диаметра;
- Выбрать лучшую предсказательную модель и уточнить диапазоны оптимальных величин.
- Если возможно, получить функциональную зависимость.


## Теоретическая часть


Определения:

- S - быстрота действия насоса, м^3/c;
- $S_{max}$ - максимальная быстрота действия насоса, м^3/c;
- T - температура газа, К;
- M - молярная масса газа, г/моль;
- $с_1 = \frac{u_2}{v_{нв}}$ - отношение окружной скорости рабочего колеса на наружном диаметре к наиболее вероятной скорости теплового движения молекул;
- $\psi = \frac{h_2}{h_1}$ - отношение толщин лопатки на внешнем и внутреннем диаметрах;
- $\theta = \frac{H_2}{H_1}$ - отношение толщин колеса на внешнем диаметре и у основания;
- $\lambda = \frac{D_1}{D_2}$ - отношение внутреннего диаметра колеса к внешнему диаметру;
- $\alpha$ - угол установки лопаток, $\degree$;
- $\frac{a_1}{b_1}$ - отношение ширины межлопаточного канала и длины лопатки у корневого сечения;

В процессе расчёта по заданной быстроте действия насоса $S = (0.9...0.95)S_{max}$ определяют наружный диаметр внешнего колеса $D_2$, м:

$$
D_2 = \sqrt{\frac{S_{max}}{18.2\pi\sqrt{\frac{T}{M}}}\cdot F(c_1, \alpha, ab_1, \lambda)},
$$

где

$$
F(c_1, \alpha, ab_1, \lambda) = \frac{\frac{a_1}{b_1}+\frac{h_1}{H_1}}{(\frac{a_1}{b_1}+\frac{h_1}{H_1}\frac{1-\lambda\psi}{1-\lambda})I_1(c_1, \alpha, ab_1, \lambda)-\lambda\frac{h_1}{H_1}\frac{1-\lambda\psi}{1-\lambda}I_2(c_1, \alpha, ab_1, \lambda)},
$$

где $I_1$ и $I_2$ - интегралы, характеризующие вероятность перехода молекул со стороны пониженного давления на сторону повышенного и в обратном направлении:

$$
I_1 = \frac{2}{\pi}\int_\lambda^1\frac{x}{ab(x)}\int_0^{ab(x)}\arcsin\Bigl(\frac{c_1x\sin{\alpha}}{\sqrt{(ab(x) - y - \cos(\alpha))^2 + \sin(\alpha)^2}}\Bigr),
$$

$$
I_2 = \frac{2}{\pi}\int_\lambda^1\frac{1}{ab(x)}\int_0^{ab(x)}\arcsin\Bigl(\frac{c_1x\sin{\alpha}}{\sqrt{(ab(x) - y - \cos(\alpha))^2 + \sin(\alpha)^2}}\Bigr),
$$

$$
ab(x) = \frac{1-\lambda}{x(\theta-1)+1-\theta\lambda}\Bigl[\frac{x}{\lambda}\frac{a_1}{b_1} + \frac{h_1}{H_1}\frac{(1-\lambda\psi)(x-\lambda)}{\lambda(1-\lambda)}\Bigr].
$$

Проверка данных интегралов показала, что для них невозможно выразить решение в элементарных функциях, а значит найти минимальный диаметр возможно только путем решения задачи оптимизации численными методами. Стоит отметить, что задача сводится к поиску минимума функции F, зависящей только от геометрии колеса, поскольку формулы сводятся к $D_2 = const\cdot\sqrt{F}$.

Для удобства дальнейших вычислений обозначим класс Wheel, описывающее колесо ступени ТМН и впоследствии будем создавать его экзепляры с различным набором параметров, характеризующих его геомерию. Оптимальное значение F будем искать, задаваясь $с_1$ и $\alpha$, находя для них также $\lambda_{опт}$ и $\Bigl(\frac{a_1}{b_1}\Bigr)_{опт}$.


## Практическая часть


In [1]:
import optuna
import numpy as np
from scipy.integrate import dblquad
from rich.progress import track

import warnings
from itertools import product

warnings.filterwarnings("ignore")
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [2]:
# example
theta, psi = 1, 1
hH_1 = 0.25

In [3]:
def ab(x, lambda_, ab_1):
    fx1 = hH_1 * (1 - lambda_ * psi) * (x - lambda_) / (lambda_ * (1 - lambda_))
    gx = (1 - lambda_) / (x * (theta - 1) + 1 - theta * lambda_)
    fx2 = x * ab_1 / lambda_
    return gx * (fx1 + fx2)

In [4]:
def i_funcs(y, x, n, lambda_, ab_1, c_1, alpha):
    num = c_1 * x * np.sin(alpha)
    den = np.sqrt((ab(x, lambda_, ab_1) - y - np.cos(alpha)) ** 2 + np.sin(alpha) ** 2)
    fx = 2 * x**n * np.arcsin(num / den)
    gx = np.pi * ab(x, lambda_, ab_1)
    return fx / gx

In [5]:
def f_obj(c_1, alpha, lambda_, ab_1):
    I_1 = dblquad(
        i_funcs,
        lambda_,
        1,
        0,
        lambda x: ab(x, lambda_, ab_1),
        args=(1, lambda_, ab_1, c_1, alpha),
    )[0]
    I_2 = dblquad(
        i_funcs,
        lambda_,
        1,
        0,
        lambda x: ab(x, lambda_, ab_1),
        args=(0, lambda_, ab_1, c_1, alpha),
    )[0]


    const = hH_1 * (1 - lambda_ * psi) / (1 - lambda_)


    det = (ab_1 + const) * I_1 - lambda_ * const * I_2


    return (ab_1 + hH_1) / det

In [6]:
def objective(trial, c_1, alpha):
    lmb = trial.suggest_float("lambda", 0.6, 0.99, step=0.01)
    ab_1 = trial.suggest_float("ab_1", 0.6, 1.2, step=0.01)
    return f_obj(c_1, alpha, lambda_=lmb, ab_1=ab_1)


In [ ]:
c_1_list = np.arange(0.2, 0.75, 0.01)
alpha_list = np.deg2rad(np.arange(10, 50, 5))
N_total = c_1_list.size * alpha_list.size
res = []
for c_1, alpha in track(product(c_1_list, alpha_list), total=N_total):
    study = optuna.create_study(direction="minimize")
    study.optimize(lambda x: objective(x, c_1, alpha), n_trials=200)
    opt_params = [c_1, alpha, study.best_params["lambda"], study.best_params["ab_1"]]
    res.append(opt_params)


In [ ]:
res

Выводы:


## Подбор моделей


Выводы:


## Общие выводы
